# State Average SAT Scores - An Observational Study

When, in 1982, average Scholastic Achievement Test (SAT) scores were first published on a state-by-state basis in the United States, the huge variation in the scores was a source of great pride for some states and of consternation for others.  Average scores ranged from a low of 790 (out of a possible 1,600) in South Carolina to a high of 1,088 in Iowa.  This 298-point spread dwarfed the 20-year national decline of 80 points.  Two researchers set out to "assess the extent to which the compositional/demographic and school-structural characteristics are implicated in SAT differences."  (Data from B. Powell and L. C. Steelman, "Variations in State SAT Performance:  Meaningful or Misleading?"  *Harvard Educational Review* 54(4) (1984): 389-412.)



The state averages of the local SAT (verbal + quantitative) scores are listed below, along with six variables that may be associated with the SAT differences among states.  Some explanatory variables come from the Powell and Steelman article, while others were obtained from the College Entrance Examination Board (by Robert Powers).  The variables are the following:  *Takers* is the percentage of the total eligible students (high school seniors) in the state who took the exam; *income* is the median income of families of test-takers, in hundreds of dollars; *years* is the average number of years that the test-takers, in hundreds of dollars; *years* is the number of years that the test-takers had formal studies in social sciences, natural sciences, and humanities; *public* is the percentage of the test-takers who attended public secondary schools; *expend* is the total state expenditure on secondary schools, expressed in hundreds of dollars per student; and *rank* is the median percentile ranking of the test-takers within their secondary school classes.

In [ ]:
# 3rd party library imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm

sns.set()
pd.options.display.float_format = "{:.3f}".format
pd.options.display.max_columns = 12

We begin by reading the data and summarizing the variables.

In [ ]:
df = pd.read_csv('case1201.csv')
df.head()

In [ ]:
df.describe()

# Display 12.4:  Matrix of scatterplots for SAT scores and explanatory variables

In [ ]:
cols = ['Rank', 'Takers', 'Years', 'Income', 'Public', 'Expend', 'SAT']
g = sns.pairplot(df[cols])

The SAT/Takers scatterplots indicate a non-linearity, so apply the logarithmic transform to the **Takers** variable.

In [ ]:
df['logTakers'] = np.log(df['Takers'])
cols.remove('Takers')
cols.insert(1, 'logTakers')
g = sns.pairplot(df[cols])

*Expend* clearly has an outlier (Alaska) and *Public* may have one (Louisiana).

# Preliminary Analysis

In [ ]:
formula = 'SAT ~ logTakers + Rank'
model = smf.ols(formula=formula, data=df).fit()
model.summary()

The takers and class rank variables explain 81% of the total variation.  Look at the effect of *Expend* on SAT scores after taking *Takers* and *Rank* into account.

## Display 12.5: Partial residual plot of state average SAT scores (adjusted for takers and median class rank) versus state expenditure

In [ ]:
fig, ax = plt.subplots(ncols=2, sharex=True, sharey=True)                      
model = smf.ols(formula='SAT ~ logTakers + Rank + Expend', data=df).fit()          
sm.graphics.plot_ccpr(model, 'Expend', ax=ax[0]) 
ax[0].set_title(rf"With Alaska: $\beta$, SE = {model.params['Expend']:.2f}, {model.bse['Expend']:.2f}")
ax[0].set_ylabel('Partial Residual')
ax[0].set_xlabel('')
                                                                                
model = smf.ols(formula='SAT ~ logTakers + Rank + Expend', data=df.query('Expend < 40')).fit()
sm.graphics.plot_ccpr(model, 'Expend', ax=ax[1])     
ax[1].set_title(rf"Without Alaska: $\beta$, SE = {model.params['Expend']:.2f}, {model.bse['Expend']:.2f}")
ax[1].set_ylabel('')
ax[1].set_xlabel('')

fig.suptitle('CCPR (partial residual plots)')
fig.supxlabel('Expenditure ($100s per student)')
fig.tight_layout()

## Partial residual plot of state average SAT scores (adjusted for takers and median class rank) versus public school

In [ ]:
fig, ax = plt.subplots(ncols=2, sharex=True, sharey=True)                      
model = smf.ols(formula='SAT ~ logTakers + Rank + Public', data=df).fit()          
sm.graphics.plot_ccpr(model, 'Public', ax=ax[0]) 
ax[0].set_title(rf"W/Louisiana: $\beta$, SE = {model.params['Public']:.2f}, {model.bse['Public']:.2f}")
ax[0].set_ylabel('Partial Residual')
ax[0].set_xlabel('')
                                                                                
model = smf.ols(formula='SAT ~ logTakers + Rank + Public', data=df.query('Public > 50')).fit()
sm.graphics.plot_ccpr(model, 'Public', ax=ax[1])     
ax[1].set_title(rf"Wo/Louisiana:  $\beta$, SE = {model.params['Public']:.2f}, {model.bse['Public']:.2f}")
ax[1].set_ylabel('')
ax[1].set_xlabel('')

fig.suptitle('CCPR (partial residual plots)')
fig.supxlabel('Test Taker Percentage in Public Schools')
fig.tight_layout()

One has to take into account the standard errors of the *Expend* slope to see that the effect of Alaska is significant, while in the *Public* case, the effect of Louisiana is not.  Are partial residual plots really necessary for that?

# Forward Variable Selection

There does not seem to be any forward selection capability in statsmodels, but it's easy to program it.

In [ ]:
# 3rd party library imports                                                     
import pandas as pd                                                             
import statsmodels.formula.api as smf                                           
from statsmodels.stats.anova import anova_lm                                    
                                                                                
                                                                                
class ForwardSelect(object):                                                    
                                                                                
    def __init__(self, df, endog):                                              
                                                                                
        self.endog = endog                                                      
        self.df = df                                                            
                                                                                
        self.current_exog = []                                                  
                                                                                
        self.candidates = df.columns.to_list()                                  
        self.candidates.remove(endog)                                           
                                                                                
    def run_forward(self):                                                      
        """                                                                     
        Go thru each new model where the candidate variables are added to the   
        new model exog vars.                                                    
        """                                                                     
                                                                                
        fresults = []                                                           
                                                                                
        for var in self.candidates:                                             
            formula = f"{self.endog} ~ {' + '.join(self.current_exog + [var])}" 
            model = smf.ols(formula=formula, data=self.df).fit()                
            atable = anova_lm(model)                                            
            fresults.append(atable.loc[var, 'F'])                               
                                                                                
        s = pd.Series(fresults, index=self.candidates)                          
        print(s)                                                                
                                                                                
    def select(self, var):                                                      
        """                                                                     
        Add a variable to the set list of exog vars, probably because of high   
        f-value.                                                                
        """                                                                     
                                                                                
        self.candidates.remove(var)                                             
        self.current_exog.append(var)
                                                                            
    def __str__(self):                                                          
        if len(self.current_exog) == 0:                                         
            formula = f"{self.endog} ~ 1"                                       
        else:                                                                   
            formula = f"{self.endog} ~ {' + '.join(self.current_exog)}"         
        model = smf.ols(formula=formula, data=self.df).fit()                    
        return str(model.summary())  

In [ ]:
data = df.query('Expend < 40').drop(['State', 'Takers'], axis='columns')
fs = ForwardSelect(data, 'SAT')
fs.run_forward()

In [ ]:
fs.select('logTakers')
fs.run_forward()

In [ ]:
fs.select('Expend')
fs.run_forward()

In [ ]:
print(fs)

Stop after selecting just two variables, *logTakers* and *Expend*.  Note that *Rank* is not chosen, and the pairplots bear this out since it is highly correlated with *logTakers*.